In [26]:
#Require Python 3.12.3
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Setup

In [27]:
import cv2
import numpy as np
from PIL import Image
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
import tempfile
import os

from diffusers import StableDiffusionPipeline
import torch
from pathlib import Path

import numpy as np

In [28]:
assts_dir= Path(os.getcwd())/"assets"


# classes

In [29]:

class VideoEditor:
    def __init__(self, fps=30, fourcc='mp4v', frame_size=(640, 480)):
        self.fps = fps
        self.frame_size = frame_size
        self.fourcc = cv2.VideoWriter_fourcc(*fourcc)
        self.temp_video_path = tempfile.mktemp(suffix='.mp4')
        self.video_writer = cv2.VideoWriter(self.temp_video_path, self.fourcc, self.fps, self.frame_size)
        self.current_time = 0  # in seconds, tracks the actual current duration of the video content
        self.audio_clips = []

        # Default settings for time text overlay
        self.default_show_time_text = False
        self.default_time_text_position = (50, 50)  # Default top-left corner
        self.default_time_text_color = (0, 255, 0)  # Default Green (B, G, R)
        self.default_time_text_font_scale = 1.0
        self.default_time_text_thickness = 2
        self.default_time_text_font = cv2.FONT_HERSHEY_SIMPLEX
        self.default_time_display_format = "HH:MM:SS.MS"

        # New class-level parameter for global display time offset
        # If None, the displayed time will default to the actual video current_time.
        # If set, it overrides the default behavior for all subsequent additions
        # where display_time_start_offset is not explicitly provided.
        self.global_display_time_offset_start = None
        # New: Counter for the displayed time when global_display_time_offset_start is active
        self.display_time_counter = 0.0

    def set_global_display_time_offset_start(self, offset_time):
        """
        Sets a global offset for the time displayed on the video.
        Any subsequent images added with show_time_text enabled and without
        an explicit display_time_start_offset will use this value as their base.

        Args:
            offset_time (float): The starting time (in seconds) to display on the video.
                                 Set to None to revert to using the actual video's current_time.
        """
        if not isinstance(offset_time, (int, float)) and offset_time is not None:
            raise TypeError("offset_time must be a number (int or float) or None.")
        if offset_time is not None and offset_time < 0:
            raise ValueError("offset_time cannot be negative.")
        self.global_display_time_offset_start = offset_time
        # Initialize the display_time_counter when the global offset is set
        self.display_time_counter = offset_time if offset_time is not None else 0.0


    def set_default_time_display_format(self, format_string):
        """
        Sets the default format for displaying time on frames.

        Args:
            format_string (str): The desired format string using placeholders:
                                 HH (hours), MM (minutes), SS (seconds), MS (milliseconds).
                                 Placeholders are case-insensitive (e.g., "hh", "mm", "ss", "ms" also work).
                                 Example: "HH:MM:SS.MS", "ss.ms", "MM:SS".
        """
        self.default_time_display_format = format_string

    def _resize_frame(self, frame):
        """
        Resizes an image frame to the video's frame_size.
        """
        return cv2.resize(frame, self.frame_size)

    def _convert_to_cv2(self, image):
        """
        Converts various image types (path, Pillow, NumPy array) to an OpenCV image (NumPy array)
        and resizes it.
        """
        if isinstance(image, str):
            img = cv2.imread(image)
            if img is None:
                raise FileNotFoundError(f"Image file not found or could not be read: {image}")
        elif isinstance(image, Image.Image):
            img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        elif isinstance(image, np.ndarray):
            img = image
        else:
            raise ValueError("Unsupported image type. Must be a file path (string), Pillow Image, or NumPy array.")
        return self._resize_frame(img)

    def _draw_time_on_frame(self, frame, time_in_sec, position, color, font_scale, thickness, font, time_format):
        """
        Draws the current time onto a video frame with a specified format.

        Args:
            frame (np.ndarray): The OpenCV image frame.
            time_in_sec (float): The time in seconds to display.
            position (tuple): (x, y) coordinates for the text.
            color (tuple): (B, G, R) color for the text.
            font_scale (float): Font scale factor.
            thickness (int): Line thickness for the text.
            font (int): OpenCV font type (e.g., cv2.FONT_HERSHEY_SIMPLEX).
            time_format (str): The format string using placeholders (HH, MM, SS, MS).
                               Placeholders are case-insensitive.
        """
        total_seconds_int = int(time_in_sec)
        milliseconds = int((time_in_sec - total_seconds_int) * 1000)
        seconds = total_seconds_int % 60
        minutes = (total_seconds_int // 60) % 60
        hours = total_seconds_int // 3600

        replacements = {
            "HH": f"{hours:02}", "hh": f"{hours:02}",
            "MM": f"{minutes:02}", "mm": f"{minutes:02}",
            "SS": f"{seconds:02}", "ss": f"{seconds:02}",
            "MS": f"{milliseconds:03}", "ms": f"{milliseconds:03}"
        }

        time_str = time_format
        for placeholder, value in replacements.items():
            time_str = time_str.replace(placeholder, value)

        cv2.putText(frame, time_str, position, font, font_scale, color, thickness, cv2.LINE_AA)
        return frame

    def add_image(self, image, duration_sec,
                  show_time_text=None, time_text_position=None, time_text_color=None,
                  time_text_font_scale=None, time_text_thickness=None, time_text_font=None,
                  display_time_start_offset=None, time_display_format=None):
        """
        Adds a single image to the video for a specified duration, with optional time display.

        Args:
            image: The image to add. Can be a file path (string), a Pillow Image object,
                   or an OpenCV image (NumPy array).
            duration_sec (float): The duration (in seconds) for which the image should be displayed.
            show_time_text (bool, optional): Whether to show the current time on the image.
                                              Defaults to self.default_show_time_text.
            time_text_position (tuple, optional): (x, y) coordinates for the text.
                                                  Defaults to self.default_time_text_position.
            time_text_color (tuple, optional): (B, G, R) color for the text.
                                               Defaults to self.default_time_text_color.
            time_text_font_scale (float, optional): Font scale factor.
                                                    Defaults to self.default_time_text_font_scale.
            time_text_thickness (int, optional): Line thickness for the text.
                                                 Defaults to self.default_time_text_thickness.
            time_text_font (int, optional): OpenCV font type.
                                            Defaults to self.default_time_text_font.
            display_time_start_offset (float, optional): The starting time (in seconds) to display
                                                         on the video for this segment.
                                                         Precedence: local param > global param > actual video current time.
            time_display_format (str, optional): The format for the time string (e.g., "HH:MM:SS.MS").
                                                 Placeholders are case-insensitive.
                                                 Precedence: local param > default class param.
        """
        frame = self._convert_to_cv2(image)
        frame_count = int(self.fps * duration_sec)

        # Resolve text display parameters
        _show_text = self.default_show_time_text if show_time_text is None else show_time_text
        _position = self.default_time_text_position if time_text_position is None else time_text_position
        _color = self.default_time_text_color if time_text_color is None else time_text_color
        _font_scale = self.default_time_text_font_scale if time_text_font_scale is None else time_text_font_scale
        _thickness = self.default_time_text_thickness if time_text_thickness is None else time_text_thickness
        _font = self.default_time_text_font if time_text_font is None else time_text_font
        _time_format = self.default_time_display_format if time_display_format is None else time_display_format

        # Determine the base time for display based on precedence
        _base_display_time_for_this_segment = self.current_time # Default fallback: actual video time

        if display_time_start_offset is not None:
            # Local override takes highest precedence
            _base_display_time_for_this_segment = display_time_start_offset
        elif self.global_display_time_offset_start is not None:
            # Global override applies if no local override. Use the continuous display_time_counter.
            _base_display_time_for_this_segment = self.display_time_counter

        for i in range(frame_count):
            # Calculate the time to display for the current frame
            current_display_time = _base_display_time_for_this_segment + (i / self.fps)
            frame_to_write = frame.copy() # Use a copy to avoid drawing on the original frame for next iteration
            if _show_text:
                frame_to_write = self._draw_time_on_frame(
                    frame_to_write, current_display_time, _position, _color, _font_scale, _thickness, _font, _time_format
                )
            self.video_writer.write(frame_to_write)

        # Always update the actual video time
        self.current_time += duration_sec

        # Only update display_time_counter if global offset is active AND no local offset was used
        if display_time_start_offset is None and self.global_display_time_offset_start is not None:
            self.display_time_counter += duration_sec


    def add_images_from_list(self, images, total_duration_sec,
                             show_time_text=None, time_text_position=None, time_text_color=None,
                             time_text_font_scale=None, time_text_thickness=None, time_text_font=None,
                             display_time_start_offset=None, time_display_format=None):
        """
        Adds several images to the video, distributing them evenly over a total duration,
        with optional time display on each image.

        Args:
            images: Can be a string (directory path), a list of strings (image file paths),
                    a list of OpenCV images (numpy arrays), or a list of Pillow images.
            total_duration_sec (float): The total duration (in seconds) that these images
                                        should occupy in the video.
            show_time_text (bool, optional): Whether to show the current time on the image.
                                              Defaults to self.default_show_time_text.
            time_text_position (tuple, optional): (x, y) coordinates for the text.
                                                  Defaults to self.default_time_text_position.
            time_text_color (tuple, optional): (B, G, R) color for the text.
                                               Defaults to self.default_time_text_color.
            time_text_font_scale (float, optional): Font scale factor.
                                                    Defaults to self.default_time_text_font_scale.
            time_text_thickness (int, optional): Line thickness for the text.
                                                 Defaults to self.default_time_text_thickness.
            time_text_font (int, optional): OpenCV font type.
                                            Defaults to self.default_time_text_font.
            display_time_start_offset (float, optional): The starting time (in seconds) to display
                                                         on the video for this segment.
                                                         Precedence: local param > global param > actual video current time.
            time_display_format (str, optional): The format for the time string (e.g., "HH:MM:SS.MS").
                                                 Placeholders are case-insensitive.
                                                 Precedence: local param > default class param.
        """
        image_list = []

        if isinstance(images, str) and os.path.isdir(images):
            for filename in sorted(os.listdir(images)):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    image_list.append(os.path.join(images, filename))
        elif isinstance(images, list):
            if all(isinstance(img, str) for img in images):
                image_list = images
            elif all(isinstance(img, (np.ndarray, Image.Image)) for img in images):
                image_list = images
            else:
                raise ValueError("List must contain only strings (paths), OpenCV images, or Pillow images.")
        else:
            raise ValueError("Unsupported 'images' type. Must be a directory path (string), a list of paths, a list of OpenCV images, or a list of Pillow images.")

        if not image_list:
            print("No images found to add.")
            return

        single_image_duration = total_duration_sec / len(image_list)

        for i, img in enumerate(image_list):
            # The display_time_start_offset passed to add_image needs to be carefully managed.
            # If the user provided a display_time_start_offset to add_images_from_list,
            # we calculate the offset for each individual image within that batch.
            # If not, we pass None, allowing add_image to use the global_display_time_offset_start
            # and its internal display_time_counter, which is the desired behavior for continuous time.
            effective_display_offset_for_image_batch = None
            if display_time_start_offset is not None:
                effective_display_offset_for_image_batch = display_time_start_offset + (i * single_image_duration)

            self.add_image(img, single_image_duration,
                           show_time_text=show_time_text,
                           time_text_position=time_text_position,
                           time_text_color=time_text_color,
                           time_text_font_scale=time_text_font_scale,
                           time_text_thickness=time_text_thickness,
                           time_text_font=time_text_font,
                           display_time_start_offset=effective_display_offset_for_image_batch, # Pass the calculated offset or None
                           time_display_format=time_display_format)

    def add_text_below_image(self, image, text, duration_sec,
                             text_box_height_ratio=0.2,
                             background_color=(0, 0, 0),
                             text_color=(255, 255, 255),
                             text_horizontal_alignment="center",
                             text_vertical_alignment="center",
                             font_scale=0.7, thickness=2, font=cv2.FONT_HERSHEY_SIMPLEX,
                             show_time_text=None, time_text_position=None, time_text_color=None,
                             time_text_font_scale=None, time_text_thickness=None, time_text_font=None,
                             display_time_start_offset=None, time_display_format=None):
        """
        Adds an image with text displayed in a box below it for a given duration.
        The combined content fits within the video's frame_size.

        Args:
            image: The input image (path, Pillow, or NumPy array).
            text (str): The text string to display.
            duration_sec (float): The duration (in seconds) for which this combined frame should be displayed.
            text_box_height_ratio (float): Ratio (0.0 to 1.0) of the frame height allocated to the text box.
            background_color (tuple): (B, G, R) color for the text box background.
            text_color (tuple): (B, G, R) color for the text.
            text_horizontal_alignment (str): Horizontal alignment of text ("left", "center", "right").
            text_vertical_alignment (str): Vertical alignment of text ("top", "center", "bottom").
            font_scale (float): Font scale factor for the text.
            thickness (int): Line thickness for the text.
            font (int): OpenCV font type (e.g., cv2.FONT_HERSHEY_SIMPLEX).
            # Parameters for optional time display (same as add_image)
            show_time_text (bool, optional): Whether to show the current time on the image.
            time_text_position (tuple, optional): (x, y) coordinates for the time text.
            time_text_color (tuple, optional): (B, G, R) color for the time text.
            time_text_font_scale (float, optional): Font scale factor for time text.
            time_text_thickness (int, optional): Line thickness for time text.
            time_text_font (int, optional): OpenCV font type for time text.
            display_time_start_offset (float, optional): The starting time to display.
            time_display_format (str, optional): The format for the time string.
        """
        if not (0.0 <= text_box_height_ratio <= 1.0):
            raise ValueError("text_box_height_ratio must be between 0.0 and 1.0.")

        original_img_frame = self._convert_to_cv2(image)
        
        video_width, video_height = self.frame_size
        text_box_height = int(video_height * text_box_height_ratio)
        image_display_height = video_height - text_box_height

        if image_display_height <= 0:
            raise ValueError("Image display height is zero or negative. Reduce text_box_height_ratio or increase frame_size.")

        resized_img_for_top = cv2.resize(original_img_frame, (video_width, image_display_height))

        text_section = np.full((text_box_height, video_width, 3), background_color, dtype=np.uint8)

        text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
        text_width, text_height_actual = text_size

        text_x, text_y = 0, 0

        if text_horizontal_alignment.lower() == "center":
            text_x = (video_width - text_width) // 2
        elif text_horizontal_alignment.lower() == "right":
            text_x = video_width - text_width - 10
        else: # Default to "left"
            text_x = 10

        if text_vertical_alignment.lower() == "center":
            text_y = (text_box_height + text_height_actual) // 2
        elif text_vertical_alignment.lower() == "bottom":
            text_y = text_box_height - 10
        else: # Default to "top"
            text_y = text_height_actual + 10

        cv2.putText(text_section, text, (text_x, text_y), font, font_scale, text_color, thickness, cv2.LINE_AA)

        final_frame = np.vstack((resized_img_for_top, text_section))

        frame_count = int(self.fps * duration_sec)

        _show_text = self.default_show_time_text if show_time_text is None else show_time_text
        _time_position = self.default_time_text_position if time_text_position is None else time_text_position
        _time_color = self.default_time_text_color if time_text_color is None else time_text_color
        _time_font_scale = self.default_time_text_font_scale if time_text_font_scale is None else time_text_font_scale
        _time_thickness = self.default_time_text_thickness if time_text_thickness is None else time_text_thickness
        _time_font = self.default_time_text_font if time_text_font is None else time_text_font
        _time_format = self.default_time_display_format if time_display_format is None else time_display_format

        _base_display_time_for_this_segment = self.current_time
        if display_time_start_offset is not None:
            _base_display_time_for_this_segment = display_time_start_offset
        elif self.global_display_time_offset_start is not None:
            _base_display_time_for_this_segment = self.display_time_counter


        for i in range(frame_count):
            current_display_time = _base_display_time_for_this_segment + (i / self.fps)
            frame_to_write = final_frame.copy()
            if _show_text:
                frame_to_write = self._draw_time_on_frame(
                    frame_to_write, current_display_time, _time_position, _time_color, _time_font_scale, _time_thickness, _time_font, _time_format
                )
            self.video_writer.write(frame_to_write)
        
        self.current_time += duration_sec

        if display_time_start_offset is None and self.global_display_time_offset_start is not None:
            self.display_time_counter += duration_sec

    def add_video(self, video_path):
        """
        Adds another video clip to the current video. The added video retains its original duration.

        Args:
            video_path (str): Path to the video file to be added.
        """
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise FileNotFoundError(f"Video file not found or could not be opened: {video_path}")

        video_fps = cap.get(cv2.CAP_PROP_FPS)
        if video_fps == 0:
            video_fps = self.fps

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = self._resize_frame(frame)
            self.video_writer.write(frame)

        duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / video_fps
        cap.release()
        self.current_time += duration
        # If global display offset is active, and we are adding a video (which doesn't have its own
        # display_time_start_offset parameter), we should also advance the display_time_counter.
        if self.global_display_time_offset_start is not None:
            self.display_time_counter += duration


    def add_audio(self, audio_path, audio_clip_start=None, audio_clip_end=None, video_start_offset=None):
        """
        Adds an audio clip to the video timeline.

        Args:
            audio_path (str): Path to the audio file.
            audio_clip_start (float, optional): The start time (in seconds) within the audio file itself.
                                                Defaults to 0 (beginning of the audio file).
            audio_clip_end (float, optional): The end time (in seconds) within the audio file itself.
                                              Defaults to the end of the audio clip.
            video_start_offset (float, optional): The time (in seconds) on the video timeline where this
                                                  audio should start. If None, it starts at the current
                                                  end time of the video (`self.current_time`).
        """
        try:
            audio_clip = AudioFileClip(audio_path)
        except Exception as e:
            raise ValueError(f"Could not load audio file {audio_path}: {e}")

        if audio_clip_start is not None or audio_clip_end is not None:
            if audio_clip_start is not None and audio_clip_end is not None and audio_clip_start > audio_clip_end:
                raise ValueError("audio_clip_start cannot be greater than audio_clip_end.")
            
            start_subclip = audio_clip_start if audio_clip_start is not None else 0
            end_subclip = audio_clip_end if audio_clip_end is not None else audio_clip.duration
            
            audio_clip = audio_clip.subclip(start_subclip, end_subclip)

        offset_on_video = video_start_offset if video_start_offset is not None else self.current_time

        self.audio_clips.append((audio_clip, offset_on_video))

    def get_video_duration(self):
        """
        Returns the current duration of the video content in seconds.
        """
        return self.current_time

    def save(self, output_path):
        """
        Finalizes the video and merges audio if present.
        """
        self.video_writer.release()

        final_clip = VideoFileClip(self.temp_video_path)

        if self.audio_clips:
            all_audios = []
            for audio, offset in self.audio_clips:
                all_audios.append(audio.set_start(offset))
            
            composite_audio = CompositeAudioClip(all_audios)
            
            final_clip = final_clip.set_audio(composite_audio)

        print(f"Saving video to {output_path}...")
        final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')
        final_clip.close()
        os.remove(self.temp_video_path)
        print("Video saved successfully and temporary file removed.")



In [30]:
class SD15ImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5", use_cuda=True, num_inference_steps=25):
        """
        Initialize the Stable Diffusion 1.5 pipeline and inference settings.
        """
        self.device = "cuda" if use_cuda and torch.cuda.is_available() else "cpu"
        self.num_inference_steps = num_inference_steps
        self.intermediate_images = []

        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            safety_checker=None
        ).to(self.device)

    def _capture_step(self, step, timestep, latents):
        """
        Internal callback to capture the image at each step.
        """
        # Decode latent to image at this step
        with torch.no_grad():
            image = self.pipe.vae.decode(latents / self.pipe.vae.config.scaling_factor).sample
            image = (image / 2 + 0.5).clamp(0, 1)
            image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
            image_pil = Image.fromarray((image * 255).astype("uint8"))
            self.intermediate_images.append(image_pil)

    def generate_image(self, prompt, negative_prompt=None, guidance_scale=7.5):
        """
        Generate image and collect intermediate steps.
        Returns a list of PIL images (one per step).
        """
        self.intermediate_images = []

        with torch.autocast(self.device) if self.device == "cuda" else torch.no_grad():
            _ = self.pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                guidance_scale=guidance_scale,
                num_inference_steps=self.num_inference_steps,
                callback=self._capture_step,
                callback_steps=1  # capture every step
            )

        return self.intermediate_images

    def save_image(self, image: Image.Image, output_path: str):
        """
        Save a single PIL image to the specified path.
        """
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        image.save(output_path)
        print(f"Image saved to {output_path}")
    def save_images(self, images, directory="generated"):
        """
        Save a list of images to the given directory.
        """
        os.makedirs(directory, exist_ok=True)
        for i, img in enumerate(images):
            path = os.path.join(directory, f"step_{i:02d}.png")
            img.save(path)
        print(f"Saved {len(images)} images to '{directory}/'")


# Generator


In [31]:
generator = SD15ImageGenerator(num_inference_steps=100)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [32]:

prompt = "Cat"
    
images = generator.generate_image(prompt)


  0%|          | 0/100 [00:00<?, ?it/s]

In [33]:


def generate_evenly_distributed_values(data):
    """
    Generates evenly distributed values for each tuple (number, start, end) in a list.

    Args:
        data (list): A list of tuples, where each tuple is (number, start, end).
                     'number' is the count of values to generate, 'start' is the
                     beginning of the range, and 'end' is the end of the range.

    Returns:
        list: A single list containing all the generated evenly distributed values.
    """
    all_values = []
    for num, start, end in data:
        # Generate 'num' evenly distributed values between 'start' and 'end'
        # np.linspace includes both start and end points
        if num > 0:
            generated_values = np.linspace(start, end, num).tolist()
            all_values.extend(generated_values)
    return all_values

# Example Usage:
# data1 = [(5, 0, 10), (3, 100, 102)]
# result1 = generate_evenly_distributed_values(data1)
# print(f"Result for data1: {result1}")
# # Expected output for data1: [0.0, 2.5, 5.0, 7.5, 10.0, 100.0, 101.0, 102.0]

# data2 = [(1, 5, 5), (4, -2, 2)]
# result2 = generate_evenly_distributed_values(data2)
# print(f"Result for data2: {result2}")
# # Expected output for data2: [5.0, -2.0, -0.6666666666666666, 0.6666666666666666, 2.0]

# data3 = []
# result3 = generate_evenly_distributed_values(data3)
# print(f"Result for data3: {result3}")
# # Expected output for data3: []

# data4 = [(0, 1, 10)]
# result4 = generate_evenly_distributed_values(data4)
# print(f"Result for data4: {result4}")
# # Expected output for data4: []


In [34]:

# Example Usage:
speed_stop=0.5
#speed_distribution = [(40, 1/30, speed_stop), (40, speed_stop, speed_stop), (19, speed_stop, 1/30)]
speed_distribution = [(40, 1/60, 1/60), (30,1/30, speed_stop),(20, speed_stop, speed_stop), (9, speed_stop, 1/30)]
result1 = generate_evenly_distributed_values(speed_distribution)
result1.append(3.202)
print(f"Result for data1: {result1}")
print(f"len {len(result1)}, sum {sum(result1)}" )
# # Expected output for data1: [0.0, 2.5, 5.0, 7.5, 10.0, 100.0, 101.0, 102.0]

Result for data1: [0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.03333333333333333, 0.04942528735632184, 0.06551724137931034, 0.08160919540229886, 0.097701149425287

In [35]:
def add_text_to_image(
    image,
    text: str,
    org: tuple[int, int] = (10, 30),  # Bottom-left corner of the text string
    font_face: int = cv2.FONT_HERSHEY_SIMPLEX,
    font_scale: float = 0.8,
    color: tuple[int, int, int] = (255,255,255),  # BGR color (Black by default)
    thickness: int = 2,
    background_color: tuple[int, int, int] = (0, 0, 0) # White background for new canvas
) -> np.ndarray:
    """
    Adds text to an image, extending the image size if the text falls outside
    the original boundaries.

    Args:
        image: The input image. Can be an OpenCV (numpy.ndarray) or Pillow (PIL.Image.Image) image.
        text (str): The text string to add.
        org (tuple[int, int]): The bottom-left corner of the text string in (x, y) coordinates.
                               Defaults to (10, 30).
        font_face (int): Font type. See cv2.FONT_HERSHEY_* for options.
                         Defaults to cv2.FONT_HERSHEY_SIMPLEX.
        font_scale (float): Font scale factor multiplied by the font-specific base size.
                            Defaults to 1.0.
        color (tuple[int, int, int]): Text color in BGR format. Defaults to (0, 0, 0) (Black).
        thickness (int): Thickness of the text lines. Defaults to 2.
        background_color (tuple[int, int, int]): Color to fill the extended canvas if the image
                                                  needs to be resized. Defaults to (255, 255, 255) (White).

    Returns:
        numpy.ndarray: The image with the added text, in OpenCV (BGR) format.
    """

    # 1. Handle Image Input: Convert Pillow image to OpenCV format if necessary
    if isinstance(image, Image.Image):
        # Convert PIL image to NumPy array (OpenCV format - BGR)
        # PIL uses RGB, OpenCV uses BGR, so convert color channels
        img_np = np.array(image)
        if img_np.ndim == 2: # Grayscale image
            img_cv = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)
        elif img_np.shape[2] == 4: # RGBA image
            img_cv = cv2.cvtColor(img_np, cv2.COLOR_RGBA2BGR)
        else: # RGB image
            img_cv = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    elif isinstance(image, np.ndarray):
        img_cv = image
        # Ensure the image is BGR (3 channels) if it's grayscale
        if img_cv.ndim == 2:
            img_cv = cv2.cvtColor(img_cv, cv2.COLOR_GRAY2BGR)
        elif img_cv.shape[2] == 4: # Handle RGBA if passed as numpy array
            img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGBA2BGR)
    else:
        raise TypeError("Input image must be a PIL Image or a NumPy array (OpenCV format).")

    # Get original image dimensions
    h_orig, w_orig = img_cv.shape[:2]

    # 2. Calculate Text Size
    # getTextSize returns ((width, height), baseline)
    # text_w is the width of the text string
    # text_h is the height of the text string (from top of ascenders to bottom of descenders)
    # baseline is the distance from the bottom-left point (org) to the baseline of the text.
    (text_w, text_h), baseline = cv2.getTextSize(text, font_face, font_scale, thickness)

    # Initialize new dimensions and offsets for the original image
    new_w, new_h = w_orig, h_orig
    offset_x, offset_y = 0, 0
    current_org_x, current_org_y = org[0], org[1]

    # 3. Determine New Image Dimensions (if text is outside)

    # Check for left extension (text starts before x=0)
    if current_org_x < 0:
        offset_x = -current_org_x
        new_w += offset_x
        current_org_x = 0 # Adjust text origin for the new canvas

    # Check for top extension (text top edge is above y=0)
    # The top of the text is at org[1] - text_h
    if current_org_y - text_h < 0:
        offset_y = -(current_org_y - text_h)
        new_h += offset_y
        current_org_y = text_h # Adjust text origin for the new canvas, so its top is at y=0

    # Check for right extension (text extends beyond original width)
    if current_org_x + text_w > new_w:
        new_w = current_org_x + text_w

    # Check for bottom extension (text extends beyond original height)
    # The bottom of the text is at org[1]
    # We also need to consider the baseline for descenders if text goes below org[1]
    # The effective bottom of the text is org[1] + baseline
    if current_org_y + baseline > new_h:
        new_h = current_org_y + baseline

    # 4. Create new canvas if needed and paste original image
    if new_w > w_orig or new_h > h_orig:
        # Create a new blank canvas with the background color
        new_image_canvas = np.full((new_h, new_w, 3), background_color, dtype=np.uint8)

        # Paste the original image onto the new canvas at the calculated offset
        new_image_canvas[offset_y : offset_y + h_orig, offset_x : offset_x + w_orig] = img_cv
        img_cv = new_image_canvas
    
    # Update the org coordinates to reflect the new canvas if it was extended
    # This is important because putText will draw relative to the current image's top-left
    adjusted_org = (current_org_x, current_org_y)

    # 5. Place Text
    cv2.putText(img_cv, text, adjusted_org, font_face, font_scale, color, thickness, cv2.LINE_AA)

    # 6. Return Image
    return img_cv


In [40]:
editor = VideoEditor(fps=30, frame_size=(512, 512))
#generator.save_images(images, directory="generated")
#editor.add_images_from_list(images, duration_sec=30)
if(len(result1) != len(images   )):
    print("time_duration and number of images are not matched.")
logo=Image.open(str(assts_dir/"AiArtStudio.AILogo.png"))
editor.add_image(logo, 3)  # Add logo for 3 seconds

editor.set_global_display_time_offset_start(0)  # Set global offset to 0 seconds
for index, duration in enumerate(result1):
   img = images[index] 
   img1=add_text_to_image(img, f"Guess what AI painting now?!",org=(5,535))
   editor.add_image(img1, duration,show_time_text=True,time_display_format="SS.MS",time_text_position=(400, 505),time_text_font_scale=0.8)

img=add_text_to_image(images[-1], "Time Up, It is a:",org=(5,250))
img=add_text_to_image(img, prompt,org=(240,350),color=(0, 0, 255), font_scale=1.2, thickness=3)
editor.add_image(img, 5,show_time_text=False)
editor.add_audio(str(assts_dir/"Long Distance.mp3"),audio_clip_end=editor.get_video_duration(),video_start_offset=0)  # Add audio starting at the beginning of the video
#editor.add_image(images[-1],3)  # Add last image for 3 seconds
editor.save("output_video1.mp4")

Saving video to output_video1.mp4...
Moviepy - Building video output_video1.mp4.
MoviePy - Writing audio in output_video1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_video1.mp4



Moviepy - Done !
Moviepy - video ready output_video1.mp4
Video saved successfully and temporary file removed.
